In [3]:
import pandas as pd
import requests
from requests import exceptions
from bs4 import BeautifulSoup
import urllib3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [4]:
urllib3.disable_warnings()

In [6]:
def create_soup(url):
    try:
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, 'lxml')
        return soup
    except:
        return 'N/A'

In [5]:
def get_specs(soup):
    try:
        body = soup.find("table",{"class":"product_detail_techinfo"}).find_all('tr')
        inner_dictionary = {}

        for row in body:
            cols = row.findChildren(recursive=False)
            cols = [ele.text.strip() for ele in cols]
            inner_dictionary.update({cols[0] : cols[1]})
        return inner_dictionary
    except:
        pass

In [97]:
def get_AllPDF(soup):
    try:
        h3_lst = soup.findAll('h3', {'class' : 'product_detail_download_list_headline'})
        header_lst = []
        for h3 in h3_lst:
            #print(h3.text)
            header_lst.append(h3.text.strip('\n\t').strip())
            h3.find_next('h3')
        #print(header_lst)
        
    
        a_lst = soup.findAll('a', {'class' : 'product_detail_download_list_item'})
        pdf_lst = []
        
        for pdf in a_lst:
            pdf_link = pdf.get('href')
            pdf_link = 'https://www.villeroy-boch.asia/' + pdf_link
            pdf_lst.append(pdf_link)
            pdf.find_next('a')
        #print(pdf_lst)
            
        pdf_dic = {}

        for i in range(len(header_lst)):
            pdf_dic.update({header_lst[i] : pdf_lst[i]})
        return pdf_dic
    
    except:
        return 'N/A'

In [161]:
def page_driver(item_id):
    
    options = webdriver.ChromeOptions()
    #driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe')
    options.headless = True
    driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe', options=options)  
        
    driver.get('https://www.villeroy-boch.asia/en/?_ga=GA1.2.1733028649.1542622446')

    print(item_id)
    try:
        popup_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll'))
        )
        popup_btn.click()


        search_icon = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'icon-search'))
        )
        search_icon.click()

        search_bar = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, 'search-form-input'))
        )
        search_bar.send_keys(item_id)

        search_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'epoq_search_button'))
        )
        search_btn.click()

        select_item = driver.find_element_by_xpath("//div[@class='epoq_resultrow epoq_grid']/a")
        item_url = select_item.get_attribute('href')
        return item_url

    except:
        pass

In [18]:
df = pd.read_excel(r'V&B_draft1.xlsx')

In [162]:
def getURLs():
    url_lst = []
    for i in df.index:
        item_url = page_driver(str(df['Item Code'][i]))
        url_lst.append(item_url)
    return url_lst

In [108]:
def get_images(soup):
    try:
        img_dic = {}
        num=1
        img_base_link = 'https://www.villeroy-boch.asia/'
        images = soup.findAll('img', {'alt' : 'Artis Surface-mounted washbasin'})
        for i in images:
            #img = i.get('src')
            img_dic.update({'image_'+str(num) : img_base_link + i.get('src')})
            num+=1
        return img_dic
    
    except:
        pass

In [10]:
def get_colors_img(soup):
    try:
        all_colors = soup.findAll('a', {'class' : 'colorchange'})
        color_img = []
        color_base_link = 'https://www.villeroy-boch.asia/'
        for i in all_colors:
            img = color_base_link + i.get('data-colorfile')
            color_img.append(img)
        
    
        all__titles = soup.findAll('img', {'class' : 'product_detail_color_img'})
        title_img = []
        for i in all__titles:
            title = i.get('title')
            title_img.append(title)
            
        color_img_dic = {}
        for i in range(len(title_img)):
            color_img_dic.update({title_img[i] : color_img[i]})

        return color_img_dic
    
    except:
        try:
            color_img = []
            color_img_dic = {}
            #if not color_img_dic:
            div = soup.findAll('div', {'class' : 'product_detail_slider_item'})
            for i in div:
                img = color_base_link + i.find_next('img').get('src')
                color_img.append(img)

            for i in range(len(color_img)):
                color_img_dic.update({'image_'+str(i+1) : color_img[i]})
            return color_img_dic
        except:
            pass

In [ ]:
urls = getURLs()
df['webpage_to_be_scraped'] = urls

In [19]:
df['soup'] = df['webpage_to_be_scraped'].apply(lambda x:create_soup(x))

In [21]:
df['img'] = df['soup'].apply(lambda x:get_colors_img(x))

In [23]:
df_temp = df['img'].apply(pd.Series)

In [24]:
df_merged = pd.concat([df, df_temp], axis=1)

In [183]:
df['All_PDF'] = df['soup'].apply(lambda x:get_AllPDF(x))

In [186]:
df_temp = df['All_PDF'].apply(pd.Series)

In [187]:
df_merged = pd.concat([df, df_temp], axis=1)

In [235]:
df_merged['specs'] =  df_merged['soup'].apply(lambda x:get_specs(x))

In [192]:
df_temp = df_merged['specs'].apply(pd.Series)

In [194]:
df_merged = pd.concat([df_merged, df_temp], axis=1)

In [43]:
df.to_excel('V&B_results.xlsx')